In [7]:
from pyspark.sql.types import StructType
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd
import math
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [8]:
my_conf = SparkConf()
my_conf.set("spark.app.name", "w12_Spark_1")
my_conf.set("spark.master", "local[*]")
my_conf.set("spark.driver.extraClassPath", "C:/Users/Vinoth/Downloads/sqljdbc_12.4.2.0_enu/sqljdbc_12.4/enu/jars/mssql-jdbc-12.4.2.jre8.jar")

spark = SparkSession.builder.config(conf = my_conf).getOrCreate()
spark


In [9]:
database_url = "jdbc:sqlserver://DESKTOP-FJ7E01E\SQLEXPRESS:1433;encrypt=true;trustServerCertificate=true"

database_name = "Learn_Sql"

jdbc_properties = {

"user": "Vinoth",

"password": "1234",

"driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"

}

In [10]:
# df_min_max = (spark.read
# .format("jdbc")
# .option("url", "jdbc:sqlserver://DESKTOP-FJ7E01E\SQLEXPRESS:1433;encrypt=true;trustServerCertificate=true;databaseName=Learn_Sql;")
# .option("query","select min(cost) as min_row, max(cost) as max_row from products")
# .option("user", "Vinoth")
# .option("password", "1234")
# .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")
# .load().collect())

# min_value, max_value = df_min_max[0][0], df_min_max[0][1]

In [11]:
# df = (spark.read
#       .option("numPartitions", 3)
#       .option("PartitionColumn", "cost")
#       .option("lowerBound", int(min_value))
#       .option("upperBound", int(max_value))
#       .jdbc(url=f"{database_url};databaseName={database_name};", table="products", properties=jdbc_properties))

In [12]:
people_df = (spark.read
      .jdbc(url=f"{database_url};databaseName={database_name};", table="people", properties=jdbc_properties))
relation_df = (spark.read
      .jdbc(url=f"{database_url};databaseName={database_name};", table="relations", properties=jdbc_properties))


In [13]:
people_df.show()

+---+---------+------+
| id|     name|gender|
+---+---------+------+
|107|     Days|    F |
|145| Hawbaker|    M |
|155|   Hansel|    F |
|202|Blackston|    M |
|227|    Criss|    F |
|278|   Keffer|    M |
|305|    Canty|    M |
|329|  Mozingo|    M |
|425|     Nolf|    M |
|534|    Waugh|    M |
|586|     Tong|    M |
|618|Dimartino|    M |
|747|    Beane|    M |
|878|  Chatmon|    F |
|904|  Hansard|    F |
+---+---------+------+



In [14]:
relation_df.show()

+----+----+
|c_id|p_id|
+----+----+
| 145| 202|
| 145| 107|
| 278| 305|
| 278| 155|
| 329| 425|
| 329| 227|
| 534| 586|
| 534| 878|
| 618| 747|
| 618| 904|
+----+----+



In [21]:
join_df = people_df.alias("t1").join(relation_df.alias("t2"), col("t1.id") == col("t2.p_id"), 'inner')

In [31]:
join_df2 = join_df.alias("t1").join(people_df.alias("t2"), col("t1.c_id") == col("t2.id") , 'inner')\
.select("t1.*", col("t2.name").alias("child_name"))\
.groupBy("child_name")\
.agg(expr("max(case when  gender = 'M' then name end) as father"),
     expr("max(case when  gender = 'F' then name end) as mother")).show()

+----------+---------+-------+
|child_name|   father| mother|
+----------+---------+-------+
| Dimartino|    Beane|Hansard|
|  Hawbaker|Blackston|   Days|
|    Keffer|    Canty| Hansel|
|   Mozingo|     Nolf|  Criss|
|     Waugh|     Tong|Chatmon|
+----------+---------+-------+

